### Test News Scope Identification Prompt with LLM Agent

In [1]:
import sys,os,time
from pathlib import Path
#import instructor
from pydantic import BaseModel
sys.path.insert(0, '../libs')
# Import SimpleLLMAgent from libs directory
PROJECT_ROOT = Path('..').resolve()
LIBS_DIR = PROJECT_ROOT / "libs"
PROMPTS_DIR = PROJECT_ROOT / "prompts"

for subdir in (PROJECT_ROOT, LIBS_DIR):
    if str(subdir) not in sys.path:
        sys.path.insert(0, str(subdir))
        
from pydantic import BaseModel, Field
# Import our general LLM factory
from llm_factory_openai import LLMAgent
from utils import read_json
from dotenv import load_dotenv

from src.run_llm_article_level import _build_batch_messages_from_articles
# Load environment variables from .env file
load_dotenv('../.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please check your .env file.")

In [2]:

# ---------------------------------------------------------------------------
# Local imports (resolved after sys.path modification)
# ---------------------------------------------------------------------------

from llm_factory_openai import BatchAsyncLLMAgent  # type: ignore
from prompt_utils import load_prompt, format_messages  # type: ignore
from utils import read_json  # type: ignore
from prompts.schemas import PROMPT_REGISTRY  # noqa: E402  (after path tweaks)

#### Load some data

In [3]:
## set up global paths 
input_data_dir = Path("//data2/CommercialData/Factiva_Repository")
test_file = input_data_dir / "2025" / "2025_articles.json"
data = read_json(test_file)

In [4]:
print(data[0]['body'])

En ese sentido ha señalado que los operadores del sistema eléctrico europeo están trabajando «no sólo para esclarecer lo ocurrido, sino también para dar apoyo en la restauración del sistema». «Por el momento el mensaje más importante es de paciencia y de seguir las instrucciones de protección civil», ha añadido Ribera. La responsable de Transición Limpia y Competencia de la Comisión Europea ha confirmado que el apagón ha afectado a España y Portugal y que ha podido causar alguna incidencia menor en el sur de Francia. Para llevar a cabo una recuperación de su sistema eléctrico, Ribera ha explicado que Portugal se ha desconectado de la red española y que España está apoyando su generación eléctrica con energía hidroeléctrica y térmica. En un comunicado, la Comisión Europea ha afirmado que se mantiene en contacto con Madrid y Lisboa y con la Red Europea de Gestores de Redes de Transporte de Electricidad para entender la causa y el impacto del apagón. La Comisión seguirá monitorizando la s

In [6]:
task="news_scope_classification"
if task not in PROMPT_REGISTRY:
    raise ValueError(f"Unknown task '{task}'. Available: {list(PROMPT_REGISTRY.keys())}")
prompt_file = PROMPT_REGISTRY[task]["prompt_file"]  # markdown template filename
response_model = PROMPT_REGISTRY[task]["response_model"]  # pydantic schema enforcing the response structure

# Resolve prompt path and load template
prompt_path = PROMPTS_DIR / prompt_file
prompt_template = load_prompt(str(prompt_path)).sections

prompt_template

{'system': 'You are an expert news text analyzer. When given a news article, you must determine whether it is primarily **Local News** or **International News**.  \n\n• **Local News**: Primarily about events, policies, people, or issues within a single country, with domestic focus (e.g., U.S. elections, state policies, local companies’ performance, U.S. sports leagues). Even if foreign aspects are mentioned, the emphasis is on domestic impacts.\n• **International News**: Primarily about relations, conflicts, policies, or events between multiple countries or focused mainly on foreign nations. Covers global affairs, foreign governments, international organizations, wars, diplomacy, cross-border trade, etc.  \n\n**Step by Step Instructions:**\n* Carefully read the news text.\n* Consider where the main **actors, events, and impacts** are located.\n* If the main focus is on domestic events (inside one country) → **Local News**.\n* If the main focus is on global affairs or multiple countries

#### Set up llm agent

In [7]:
netmind_api_key = os.getenv("Netmind_api_key")
if not netmind_api_key:
    raise ValueError("Netmind_api_key not found in environment variables. Please check your .env file.")

In [8]:
local_model_args = {"model":"Qwen/Qwen3-8B",
                    "base_url":"https://api.netmind.ai/inference-api/openai/v1",
                    "temperature":0.0,
                    "api_key":netmind_api_key
                    }
openai_agent = LLMAgent(**local_model_args)

result = openai_agent.test_connection()
print(f"OpenAI test result: {result}")

OpenAI test result: Hello! Yes, I'm here and ready to help. How can I assist you today? 😊


In [9]:
from prompts.schemas import NewsScopeClassificationResponse

In [10]:
batch_messages, batch_ids = _build_batch_messages_from_articles(data[:100], prompt_template,max_article_length=1000)
batch_messages[0]

[{'role': 'system',
  'content': 'You are an expert news text analyzer. When given a news article, you must determine whether it is primarily **Local News** or **International News**.  \n\n• **Local News**: Primarily about events, policies, people, or issues within a single country, with domestic focus (e.g., U.S. elections, state policies, local companies’ performance, U.S. sports leagues). Even if foreign aspects are mentioned, the emphasis is on domestic impacts.\n• **International News**: Primarily about relations, conflicts, policies, or events between multiple countries or focused mainly on foreign nations. Covers global affairs, foreign governments, international organizations, wars, diplomacy, cross-border trade, etc.  \n\n**Step by Step Instructions:**\n* Carefully read the news text.\n* Consider where the main **actors, events, and impacts** are located.\n* If the main focus is on domestic events (inside one country) → **Local News**.\n* If the main focus is on global affairs

In [31]:
for messages in batch_messages:
    print("----------------------------")
    print("----------------------------")
    structured_result = openai_agent.get_response_content(messages, response_format=NewsScopeClassificationResponse)
    print("Structured output label:", structured_result.classification)
    print("Structured output Justification:", structured_result.justification)
    print("-------")
    print(messages[1]['content'])
    print("\n\n")
    time.sleep(1)
        

----------------------------
----------------------------
Structured output label: International News
Structured output Justification: The text discusses a power outage affecting Spain and Portugal, with the European Commission (Brussels) involved in coordination. While the event is in Spain and Portugal, the focus is on international cooperation and the European Union's response, making it international in scope.
-------
Classify the following text into Local News or International News:
Bruselas asegura que «no hay indicios de un boicot o un ciberataque» Bruselas tiene su vista puesta en el apagón que ha afectado este lunes a España y Portugal. La vicepresidenta de la Comisión Europea, Teresa Ribera, ha pedido máxima ante este incidente y ha asegurado que, por el momento, «no hay nada que nos permita afirmar que ha habido un boicot o un ciberataque». El Ejecutivo comunitario está en contacto con las autoridades de los dos países para dar seguimiento y «apoyo», en un momento en el que 